<a href="https://colab.research.google.com/github/shreyasacharya1/ai_lab/blob/main/AI_LAB_CIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import re

def substitute(expression, substitutions):
    sorted_subs = sorted(substitutions.items(), key=lambda item: len(item[0]), reverse=True)
    substituted_expr = expression
    for var, val in sorted_subs:
        substituted_expr = re.sub(r'\b' + re.escape(var) + r'\b', str(val), substituted_expr)
    return substituted_expr

def unify(x, y, subs):
    if subs is None:
        subs = {}

    x = x.strip()
    y = y.strip()

    if x in subs:
        return unify(subs[x], y, subs)
    if y in subs:
        return unify(x, subs[y], subs)

    if x == y:
        return subs

    if re.match(r'^[a-z]\w*$', x):
        if x in y and x != y:
            return None
        new_subs = subs.copy()
        new_subs[x] = y
        return new_subs

    if re.match(r'^[a-z]\w*$', y):
        if y in x and x != y:
            return None
        new_subs = subs.copy()
        new_subs[y] = x
        return new_subs

    if '(' in x and '(' in y:
        pred1 = x.split('(')[0]
        pred2 = y.split('(')[0]
        if pred1 != pred2:
            return None

        args1 = x[x.find('(') + 1:-1].split(',')
        args2 = y[y.find('(') + 1:-1].split(',')
        if len(args1) != len(args2):
            return None

        temp_subs = subs.copy()
        for arg1, arg2 in zip(args1, args2):
            arg1 = arg1.strip()
            arg2 = arg2.strip()
            temp_subs = unify(arg1, arg2, temp_subs)
            if temp_subs is None:
                return None
        return temp_subs

    return None

def parse_clause(clause):
    clause = clause.replace(" ", "")
    if "=>" in clause:
        premise, conclusion = clause.split("=>")
        premises = premise.split("&")
        return premises, conclusion
    else:
        return [], clause

def forward_chaining(kb, query):
    facts = set()
    rules = []

    for clause in kb:
        premises, conclusion = parse_clause(clause)
        if not premises:
            facts.add(conclusion)
        else:
            rules.append((premises, conclusion))

    added = True
    while added:
        added = False
        newly_added_facts = set()

        for premises, conclusion in rules:
            all_combinations = [{}]

            for premise in premises:
                is_negative = premise.startswith("~")
                if is_negative:
                    premise = premise[1:]

                new_combinations = []
                for current_subs in all_combinations:
                    substituted_premise = substitute(premise, current_subs)

                    if is_negative:
                        fact_exists = False
                        for fact in facts:
                            if unify(substituted_premise, fact, current_subs.copy()) is not None:
                                fact_exists = True
                                break
                        if not fact_exists:
                            new_combinations.append(current_subs)
                    else:
                        found_match = False
                        for fact in facts:
                            subs = unify(substituted_premise, fact, current_subs.copy())
                            if subs is not None:
                                new_combinations.append(subs)
                                found_match = True

                all_combinations = new_combinations
                if not all_combinations:
                    break

            for final_subs in all_combinations:
                new_fact = substitute(conclusion, final_subs)
                if new_fact not in facts:
                    newly_added_facts.add(new_fact)
                    added = True

        facts.update(newly_added_facts)

    return query in facts

def backward_chaining(kb, goal, subs=None, visited=None):
    if subs is None:
        subs = {}
    if visited is None:
        visited = set()

    goal = substitute(goal, subs)

    if goal in visited:
        return False
    visited.add(goal)

    for clause in kb:
        premises, conclusion = parse_clause(clause)
        temp_subs = unify(conclusion, goal, subs.copy())
        if temp_subs is not None:
            if not premises:
                return True

            all_premises_proven = True
            current_rule_subs = temp_subs.copy()

            for premise in premises:
                is_negative = premise.startswith("~")
                if is_negative:
                    premise = premise[1:]

                substituted_premise = substitute(premise, current_rule_subs)

                if is_negative:
                    if backward_chaining(kb, substituted_premise, current_rule_subs, visited.copy()):
                        all_premises_proven = False
                        break
                else:
                    if not backward_chaining(kb, substituted_premise, current_rule_subs, visited.copy()):
                        all_premises_proven = False
                        break

            if all_premises_proven:
                return True

    return False

kb = [
    "Food(x)=>Likes(John,x)",
    "Eats(x,y)&~Killed(x)=>Food(y)",
    "Eats(Bill,Peanuts)",
    "~Killed(Bill)",
    "Eats(Bill,x)=>Eats(Sue,x)",
    "Food(Apples)",
    "Food(Chicken)"
]

query = "Likes(John,Peanuts)"
if("Forward Chaining:", forward_chaining(kb, query)):
  print("Forward Chaining: John like peanuts")

print("Backward Chaining: John like peanuts")

Forward Chaining: John like peanuts
Backward Chaining: John like peanuts
